### Application logger

In [1]:
from datetime import datetime

class App_logger:
    def __init__(self):
        pass
    
    def log(self,file_object,log_message):
        self.now=datetime.now()
        self.date = self.now.date()
        self.current_time = self.now.strftime('%H:%M:%S')
        file_object.write(str(self.date)+'/'+str(self.current_time)+'\t\t'+log_message+'\n')

In [2]:
App_logger()

## File operations

### Files methods

In [3]:
import pickle
import os
import shutil

class File_Operations: # this class shall be used to save model after training and load the saved model for predictions
    
    def __init__(self,file_object,logger_object):
        self.file_object = file_object
        self.logger_object = logger_object
        self.model_directory = 'D:/Gautam/Downloads/Wafer/models'
        
    def save_model(self,model,filename): # this method saves the model file to the directory
        
        self.logger_object.log(self.file_object,'Entered the save_model method in File_Operation class')
        
        try:
            path = os.path.join(self.model_directory,filename) # create separate directory for each cluster
            if os.path.isdir(path): #remove previously existing models for each clusters
                shutil.rmtree(self.model_directory)
                os.makedirs(path)
                
            else:
                os.makedirs(path)
            with open(path + '/'+filename+'.sav','wb') as f:
                pickle.dump(model, f) # save the model to file
            
            self.logger_object.log(self.file_object,'Model File'+filename+'saved.Exited the save_model method of the Model_Finder class')
            return 'success'
        
        except Exception as e:
            self.logger_object.log(self.file_object,'Exception occured in save_model method of the Model_Finder class. Exception message:  ' + str(e))
            self.logger_object.log(self.file_object,'Model File '+filename+' could not be saved. Exited the save_model method of the Model_Finder class')
            raise Exception()
    
    def load_model(self,filename): # this method is for the model file loaded in memory
        
        self.logger_object.log(self.file_object,'Entered the load_model method of the File_Operation class')
        
        try:
            with open(self.model_directory + filename + '/' + filename + '.sav','rb') as f:
                self.logger_object.log(self.file_object,'Model File' + filename + 'loaded. Exited the load_model method of the Model_Finder class')
                return pickle.load(f)
        
        except Exception as e:
            
            self.logger_object.log(self.file_object,'Exception occured in load_model method of the Model_Finder class. Exception message:  ' + str(e))
            self.logger_object.log(self.file_object,'Model File ' + filename + ' could not be saved. Exited the load_model method of the Model_Finder class')
            raise Exception()
    
    def find_correct_model_file(self,cluster_number): # Select the correct model based on cluster number
        
        self.logger_object.log(self.file_object,'Entered the find_correct_model_file method of the File_Operation class')
        
        try:
            self.cluster_number = cluster_number
            self.folder_name = self.model_directory
            self.list_of_model_files = []
            self.list_of_files = os.listdir(self.folder_name)
            for self.file in self.list_of_files:
                try:
                    if (self.file.index(str( self.cluster_number))!=-1):
                        self.model_name=self.file
                except:
                    continue
            self.model_name=self.model_name.split('.')[0]
            self.logger_object.log(self.file_object,'Exited the find_correct_model_file method of the Model_Finder class.')
            return self.model_name
        
        except Exception as e:
            self.logger_object.log(self.file_object,'Exception occured in find_correct_model_file method of the Model_Finder class. Exception message:  ' + str(e))
            
            self.logger_object.log(self.file_object,'Exited the find_correct_model_file method of the Model_Finder class with Failure')
            raise Exception()            

## Data ingestion

### Data loader for training 

In [4]:
# writing function for reading file and logging it

class Data_Getter:
    
    def __init__(self,file_object,logger_object): # this method is for obtaining data from source
        self.training_file=r'C:\Users\Gautam\Desktop\waferFaultDetection\code\WaferFaultDetection_new\Training_FileFromDB\InputFile.csv'
        self.file_object=file_object
        self.logger_object=logger_object
    
    def get_data(self): # this method is for reading data from source into dataframe
        
        self.logger_object.log(self.file_object,'Entered the get data method of the Data_getter class')
        try:
            self.df = pd.read_csv(self.training_file)
            self.logger_object.log(self.file_object,'Data loaded successful exited the get data method')
            return self.df
        except Exception as e:
            self.logger_object.log(self.file_object,'Exception occured in get_data method of Data_getter class Exception is: '+str(e))
            self.logger_object.log(self.file_object,'Data Load Unsuccessful.Exited the get_data method of the Data_Getter class')
            
            raise Exception()
        
        

### Data loader for prediction

In [5]:
import pandas as pd

class Data_Getter_Pred:
    
    def __init__(self, file_object, logger_object):
        self.prediction_file=r'C:\Users\Gautam\Desktop\waferFaultDetection\code\WaferFaultDetection_new\Prediction_FileFromDB\InputFile.csv'
        self.file_object=file_object
        self.logger_object=logger_object

    def get_data(self):
        
        self.logger_object.log(self.file_object,'Entered the get_data method of the Data_Getter class')
        try:
            self.data= pd.read_csv(self.prediction_file) # reading the data file
            self.logger_object.log(self.file_object,'Data Load Successful.Exited the get_data method of the Data_Getter class')
            return self.data
        except Exception as e:
            self.logger_object.log(self.file_object,'Exception occured in get_data method of the Data_Getter class. Exception message: '+str(e))
            self.logger_object.log(self.file_object,'Data Load Unsuccessful.Exited the get_data method of the Data_Getter class')
            raise Exception()


## Data Preprocessing

### Clustering

In [6]:
## we write function for reading the data with logging

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from kneed import KneeLocator


class KMeansClustering:
    
    def __init__(self,file_object,logger_object):
        self.file_object = file_object
        self.logger_object = logger_object
        
    def elbow_plot(self,data):  # his method saves the plot to decide the optimum number of clusters to the file.
        
        self.logger_object.log(self.file_object,'Entered the elbow_plot method of the KMeans Clustering class')
        wcss = []
        try:
            for i in range(1,11):
                kmeans = KMeans(n_clusters=i,init='k-means++',random_state=42)
                kmeans.fit(data)
                wcss.append(kmeans.inertia_)
            
            plt.plot(range(1,11),wcss)
            #plt.show()
            plt.savefig('D:/Gautam/Downloads/Wafer/preprocessing_data/K-Means_Elbow.PNG')
            
            # finding the value of the optimum cluster programmatically
            self.kn  = KneeLocator(range(1,11),wcss,curve='convex',direction='decreasing')
            self.logger_object.log(self.file_object,'The optimum number of clusters is '+str(self.kn.knee)+'Exited the elbow plot method')
            return self.kn.knee
        except Exception as e:
            self.logger_object.log(self.file_object,'Exception occured in elbow_plot method of the KMeansClustering class. Exception message:  ' + str(e))
            self.logger_object.log(self.file_object,'Finding the number of clusters failed. Exited the elbow_plot method of the KMeansClustering class')
            raise Exception()
            
            
    def create_clusters(self,data,number_of_clusters):
        self.logger_object.log(self.file_object,'Entered the create_clusters method of KMeans clustering class')
        self.data = data
        
        try:
            self.kmeans = KMeans(n_clusters=number_of_clusters,init='k-means++',random_state=42)
            self.logger_object.log(self.file_object,'printing object of kmeans'+str(self.kmeans))
            
            # divide data into clusters
            self.y_kmeans = self.kmeans.fit_predict(data)
            self.file_op = File_Operations(self.file_object,self.logger_object)
            
            # saving the KMeans model to directory
            # passing 'Model' as the functions need three parameters
            self.save_model = self.file_op.save_model(self.kmeans,'KMeans')
            
            # creating new column in dataset for storing the cluster information
            self.data['Cluster'] = self.y_kmeans  
            self.logger_object.log(self.file_object,'succesfully created'+str(self.kn.knee)+'clusters.Exited the create cluster method of class KMeansClustering class')
            return self.data
        except Exception as e:
            self.logger_object.log(self.file_object,'Exception occured in create_clusters method of the KMeansClustering class. Exception message:  ' + str(e))
            self.logger_object.log(self.file_object,'Fitting the data to clusters failed. Exited the create_clusters method of the KMeansClustering class')
            raise Exception()        

### preprocessing

In [7]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

class Preprocessor:
    
    def __init__(self,file_object,logger_object):
        self.file_object = file_object
        self.logger_object = logger_object
        
    def remove_columns(self,data,columns): # removes the required colums from dataframe
        
        self.logger_object.log(self.file_object,'Entered the remove_colunms method of Preprocessor class')
        self.data=data
        self.columns=columns
        
        try:
            # drop the labels specified in the columns
            self.useful_data = self.data.drop(labels=self.columns,axis=1)
            self.logger_object.log(self.file_object,'Exited the remove columns method of preprocessing class')
            return self.useful_data
        
        except Exception as e:
            self.logger_object.log(self.file_object,'Exception occured in remove_columns method of the Preprocessor class. Exception message:  '+str(e))
            self.logger_object.log(self.file_object,'Exited the remove columns method of preprocessing class')
            raise Exception()
            
            
    def separate_label_features(self,data,label_column_name): # seperates the features and labels 
        
        self.logger_object.log(self.file_object,'Entered the separate_label_features of Preprocessor class')        
        try: 
            self.X = data.drop(labels=label_column_name,axis=1) # seperating features 
            self.y = data[label_column_name]                    # Filter the label column
            self.logger_object.log(self.file_object,'Exited the separate_label_features of preprocessor class')
            return self.X,self.y
        except Exception as e:
            self.logger_object.log(self.file_object,'Exception occured in separate_label_feature method of the Preprocessor class. Exception message:  ' + str(e))
            self.logger_object.log(self.file_object, 'Label Separation Unsuccessful. Exited the separate_label_feature method of the Preprocessor class')
            raise Exception()
            
            
    def is_null_present(self,data): # checks weather there is a null value or not in pandas dataframe
        
        self.logger_object.log(self.file_object,'Entered the is_null_present method in Preprocessor class')
        self.null_present=False
        try:
            self.null_counts = self.data.isna().sum()
            for i in self.null_counts:
                if i>0:
                    self.null_present=True
                    break
            
            # write the logs to see which columns have null values 
            # and store the null columns
            if(self.null_present):
                
                dataframe_with_null = pd.DataFrame()
                #dataframe_with_null['columns'] = data.columns
                dataframe_with_null['missing_value_count'] = np.asarray(self.data.isna().sum())

                # storing the null values
                dataframe_with_null.to_csv('D:/Gautam/Downloads/Wafer/preprocessing_data/null_values.csv')

            self.logger_object.log(self.file_object,'Finding the null values is done,stored in the null value folder,Exited the is_null_present method in preprocessing class')
            return self.null_present
        
        except Exception as e:
            self.logger_object.log(self.file_object,'Exception occured in is_null_present method of the Preprocessor class. Exception message:  ' + str(e))
            self.logger_object.log(self.file_object,'Finding missing values failed. Exited the is_null_present method of the Preprocessor class')
            raise Exception()
            
    def impute_missing_values(self,data): # this method imputes all missing values in dataframe with KNN-imputer
        
        self.logger_object.log(self.file_object,'Entered the impute_missing_values method of preprocessing class')
        try:
            imputer = KNNImputer(n_neighbors=3,weights='uniform',missing_values=np.nan)
            self.new_array = imputer.fit_transform(self.data)
            
            # convert ndarray to a dataframe
            self.new_data = pd.DataFrame(data=self.new_array)#columns = data.columns)
            self.logger_object.log(self.file_object,'Imputed succefull, Exited the impute_missing_values method of preprocessing class')
            return self.new_data
        
        except Exception as e:
            self.logger_object.log(self.file_object,'Exception occured in impute_missing_values method of the Preprocessor class. Exception message:  ' + str(e))
            self.logger_object.log(self.file_object,'Imputing missing values failed. Exited the impute_missing_values method of the Preprocessor class')
            raise Exception()
            
    def get_columns_with_zero_std_deviation(self,data): # this method is used to find out with column has zero standard deviation
        
        self.logger_object.log(self.file_object,'Entered the get_columns_with_zero_std_deviation')
        
        self.columns = data.columns
        self.data_n = data.describe()
        self.col_to_drop=[]
        
        try:
            for x in self.columns:
                if (self.data_n[x]['std']==0):
                    self.col_to_drop.append(x)
            self.logger_object.log(self.file_object,'Column search for standard deviatoin of zero successfu;,Exited get_columns with zero std in class preprocessing')
            return self.col_to_drop
        
        except Exception as e:
            self.logger_object.log(self.file_object,'Exception occured in get_columns_with_zero_std_deviation method of the Preprocessor class. Exception message:  ' + str(e))
            self.logger_object.log(self.file_object, 'Column search for Standard Deviation of Zero Failed. Exited the get_columns_with_zero_std_deviation method of the Preprocessor class')
            raise Exception()      

## Best Model Tuner

### tuner

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

class Model_Finder: # this class be used to find the model with best accuracy and AUC score
    
    def __init__(self,file_object,logger_object):
        self.file_object = file_object
        self.logger_object = logger_object
        self.clf = RandomForestClassifier()
        self.xgb = XGBClassifier(objective='binary:logistic')
        
    def get_best_params_for_random_forest(self,train_x,train_y):
        
        self.logger_object.log(self.file_object,'Entered the get_best_params_for_random_forest in Model_Finder Class')
        try:
            # initializing with different combinations of parameters
            self.param_grid = {"n_estimators": [10, 50, 100, 130], "criterion": ['gini', 'entropy'],
                               "max_depth": range(2, 4, 1), "max_features": ['auto', 'log2']}
            
            self.grid = GridSearchCV(estimator = self.clf,param_grid=self.param_grid, cv=5, verbose=3)
            
            # finding the best parameters
            self.grid.fit(train_x, train_y)
            
            #extracting the best paramters
            self.criterion = self.grid.best_params_['criterion']
            self.max_depth = self.grid.best_params_['max_depth']
            self.max_features = self.grid.best_params_['max_features']
            self.n_estimators = self.grid.best_params_['n_estimators']
            
            
            # creating a new model with best parameters
            self.clf  = RandomForestClassifier(n_estimators = self.n_estimators, criterion = self.criterion,max_depth = self.max_depth, max_features = self.max_features)
            # training the mew model
            self.clf.fit(train_x, train_y)
            self.logger_object.log(self.file_object,'Random Forest best params: '+str(self.grid.best_params_)+'. Exited the get_best_params_for_random_forest method of the Model_Finder class')
            
            return self.clf
                
            
        except Exception as e:
            self.logger_object.log(self.file_object,'Exception occured in get_best_params_for_random_forest method of the Model_Finder class. Exception message:  ' + str(e))
            self.logger_object.log(self.file_object,'Random Forest Parameter tuning  failed. Exited the get_best_params_for_random_forest method of the Model_Finder class')
            raise Exception()
        
    def get_best_params_for_xgboost(self,train_x,train_y):
        
        self.logger_object.log(self.file_object,'Entered the get_best_params_for_xgboost in Model_Finder Class')
        
        try:
            
            # initializing with different combinations of parameters
            self.param_grid_xgboost = {'learning_rate': [0.5, 0.1, 0.01, 0.001],
                                       'max_depth': [3, 5, 10, 20],'n_estimators': [10, 50, 100, 200]}
            
            self.grid = GridSearchCV(estimator = self.xgb,param_grid=self.param_grid_xgboost, cv=5, verbose=3)
            
            # finding the best parameters
            self.grid.fit(train_x, train_y)
            
            # extracting the best parameters
            self.learning_rate = self.grid.best_params_['learning_rate']
            self.max_depth = self.grid.best_params_['max_depth']
            self.n_estimators = self.grid.best_params_['n_estimators']

            # creating a new model with the best parameters
            self.xgb = XGBClassifier(learning_rate=self.learning_rate, max_depth=self.max_depth, n_estimators=self.n_estimators)
            # training the mew model
            self.xgb.fit(train_x, train_y)
            self.logger_object.log(self.file_object,'XGBoost best params: ' + str(self.grid.best_params_) + '. Exited the get_best_params_for_xgboost method of the Model_Finder class')
            return self.xgb
        
        except Exception as e:
            
            self.logger_object.log(self.file_object,'Exception occured in get_best_params_for_xgboost method of the Model_Finder class. Exception message:  ' + str(e))
            self.logger_object.log(self.file_object,'XGBoost Parameter tuning  failed. Exited the get_best_params_for_xgboost method of the Model_Finder class')
            
            raise Exception()    
            
            
    def get_best_model(self,train_x,train_y,test_x,test_y):
        
        self.logger_object.log(self.file_object,'Entered the get_best_model method in Model_Finder class')
        
    
        try:
            # create best model for XGBoost
            
            self.xgboost = self.get_best_params_for_xgboost(train_x,train_y)
            self.prediction_xgboost = self.xgboost.predict(test_x)
            
            if len(test_y.unique())==1: #if there is only one label in y, then roc_auc_score returns error. We will use accuracy in that case
                self.xgboost_score = accuracy_score(test_y,self.prediction_xgboost)
                self.logger_object.log(self.file_object,'Accuracy score of xgboost is '+str(self.xgboost_score)) # Log AUC
            
            else:
                self.xgboost_score = roc_auc_score(test_y,self.prediction_xgboost) # AUC for XGBOOST
                self.logger_object.log(self.file_object,'Accuracy score of xgboost is '+str(self.xgboost_score)) # Log AUC
            
            
            # create best model for RandomForest
            
            self.random_forest=self.get_best_params_for_random_forest(train_x,train_y)
            self.prediction_random_forest=self.random_forest.predict(test_x) # prediction using the Random Forest Algorithm
            
            if len(test_y.unique()) == 1:#if there is only one label in y, then roc_auc_score returns error. We will use accuracy in that case
                self.random_forest_score = accuracy_score(test_y,self.prediction_random_forest)
                self.logger_object.log(self.file_object,'Accuracy score of RF is '+str(self.random_forest_score)) # Log AUC
            else:
                self.random_forest_score = roc_auc_score(test_y, self.prediction_random_forest) # AUC for Random Forest
                self.logger_object.log(self.file_object,'Accuracy score of RF is '+str(self.random_forest_score))
                
            # comparing two models
            if(self.random_forest_score <  self.xgboost_score):
                return 'XGBOOST',self.xgboost
            else:
                return 'RandomForest', self.random_forest
            
        except Exception as e:
            self.logger_object.log(self.file_object,'Exception occured in get_best_model method of the Model_Finder class. Exception message:  ' + str(e))
            self.logger_object.log(self.file_object,'Model Selection Failed. Exited the get_best_model method of the Model_Finder class')
            raise Exception()
            
             

## Training model

In [9]:
from sklearn.model_selection import train_test_split
#from data_ingestion import data_loader
#from data_preprocessing import preprocessing
#from data_preprocessing import clustering
#from best_model_finder import tuner
#from file_operations import file_methods
#from application_logger import logger


class trainModel:
    
    def __init__(self):
        self.log_writer = App_logger()
        self.file_object = open('D:/Gautam/Downloads/Wafer/training_logs','a+')
        
    def trainingModel(self):
        self.log_writer.log(self.file_object,'Entered the training')
        
        try:
            # Getting data from source
            data_getter = Data_Getter(self.file_object,self.log_writer)
            data = data_getter.get_data()
            self.log_writer.log(self.file_object,'Successfully got data')
            
            # preprocessing
            preprocessor = Preprocessor(self.file_object,self.log_writer)
            data = preprocessor.remove_columns(data,['Wafer'])
            self.log_writer.log(self.file_object,'Successfully preprocessed')
            
            # creating features and labels
            X, y=preprocessor.separate_label_features(data,label_column_name='Output')
            self.log_writer.log(self.file_object,'Successfully created features')
            
            # check if missing values are present in the dataset
            is_null_present=preprocessor.is_null_present(X)
            self.log_writer.log(self.file_object,'Successfully checked missing values')
            
            # if missing values are present in the dataset,replace them 
            if(is_null_present):
                X=preprocessor.impute_missing_values(X)
            self.log_writer.log(self.file_object,'Successfully imputed missing values')
                
            # check further which columns do not contribute to prediction
            # if the standard deviation for a column is zero,it means column has constant values
            # and they are giving the same output both for good and bad sensors
            # prepare the list of such columns to drop
            
            cols_to_drop = preprocessor.get_columns_with_zero_std_deviation(X)
            self.log_writer.log(self.file_object,'successfully got columns we want to drop')
            
            #drop the columns obtained above
            X = preprocessor.remove_columns(X,cols_to_drop)
            self.log_writer.log(self.file_object,'Successfully dropped columns')
            
            # Apply clustering approch
            kmeans = KMeansClustering(self.file_object,self.log_writer) # object intialization for kmeansclustering class
            number_of_clusters = kmeans.elbow_plot(X) # using elbow plot to find the number of optimum clusters
            self.log_writer.log(self.file_object,'Successfully plotted elbow curve and got number_of_clusters as '+str(number_of_clusters))
            
            # Divide the data into clusters
            X = kmeans.create_clusters(X,number_of_clusters)
            self.log_writer.log(self.file_object,'Successfully Divided the data')
            
            # creating new column consisting corresponding cluster assignment
            X['Labels'] = y
            
            # getting the unique clusters from our dataset
            list_of_clusters = X['Cluster'].unique()
            self.log_writer.log(self.file_object,'Successfully got unique Clusters')
            
            # parsing all the clusters and looking for the best ml algorithm for each cluster
            
            for i in list_of_clusters:
                #filter the data for one cluster
                cluster_data = X[X['Cluster']==i]
                
                # prepare the feature and label columns
                cluster_features = cluster_data.drop(['Labels','Cluster'],axis=1)
                cluster_label = cluster_data['Labels']
                
                # splitting the data into train and test for each cluster one by one
                
                x_train,x_test,y_train,y_test = train_test_split(cluster_features,cluster_label,test_size=0.25,random_state=355)
                
                model_finder = Model_Finder(self.file_object,self.log_writer) # object initialization
                
                # getting the best model for each clusters
                best_model_name, best_model = model_finder.get_best_model(x_train,y_train,x_test,y_test)
                
                # saving the best model for each of the clusters
                file_op = File_Operations(self.file_object,self.log_writer) # object initialization
                save_model = file_op.save_model(best_model,best_model_name+str(i))
            
            # logging the succesful training
            self.log_writer.log(self.file_object,'Successful End of training')
            self.file_object.close()
            
        except Exception as e:
            # logging the unsuccessful Training
            self.log_writer.log(self.file_object, 'Unsuccessful End of Training '+str(e))
            self.file_object.close()
            raise Exception    

## Predicting from model

In [10]:
import pandas
#from file_operations import file_methods
#from data_preprocessing import preprocessing
#from data_ingestion import data_loader_prediction
#from application_logging import logger
#from Prediction_Raw_Data_Validation.predictionDataValidation import Prediction_Data_validation


class prediction:

    def __init__(self)#path):
        self.file_object = open("D:/Gautam/Downloads/Wafer/Prediction_Log.txt", 'a+')
        self.log_writer = App_logger()
        #if path is not None:
            #self.pred_data_val = Prediction_Data_validation(path)

    def predictionFromModel(self):

        try:
            #self.pred_data_val.deletePredictionFile() #deletes the existing prediction file from last run!
            self.log_writer.log(self.file_object,'Start of Prediction')
            data_getter = Data_Getter_Pred(self.file_object,self.log_writer)
            data=data_getter.get_data()

            #code change
            # wafer_names=data['Wafer']
            # data=data.drop(labels=['Wafer'],axis=1)

            preprocessor=Preprocessor(self.file_object,self.log_writer)
            is_null_present=preprocessor.is_null_present(data)
            if(is_null_present):
                data=preprocessor.impute_missing_values(data)

            cols_to_drop=preprocessor.get_columns_with_zero_std_deviation(data)
            data=preprocessor.remove_columns(data,cols_to_drop)
            #data=data.to_numpy()
            file_loader=File_Operations(self.file_object,self.log_writer)
            kmeans=file_loader.load_model('KMeans')

            ##Code changed
            #pred_data = data.drop(['Wafer'],axis=1)
            clusters=kmeans.predict(data.drop(['Wafer'],axis=1))#drops the first column for cluster prediction
            data['clusters']=clusters
            clusters=data['clusters'].unique()
            for i in clusters:
                cluster_data= data[data['clusters']==i]
                wafer_names = list(cluster_data['Wafer'])
                cluster_data=data.drop(labels=['Wafer'],axis=1)
                cluster_data = cluster_data.drop(['clusters'],axis=1)
                model_name = file_loader.find_correct_model_file(i)
                model = file_loader.load_model(model_name)
                result=list(model.predict(cluster_data))
                result = pandas.DataFrame(list(zip(wafer_names,result)),columns=['Wafer','Prediction'])
                path="D:/Gautam/Downloads/Wafer/Predictions.csv"
                result.to_csv("D:/Gautam/Downloads/Wafer/Predictions.csv",header=True,mode='a+') #appends result to prediction file
            self.log_writer.log(self.file_object,'End of Prediction')
        except Exception as ex:
            self.log_writer.log(self.file_object, 'Error occured while running the prediction!! Error:: %s' % ex)
            raise ex
        return path, result.head().to_json(orient="records")






## Main 

In [ ]:
from wsgiref import simple_server
from flask import Flask, request, render_template
from flask import Response
import os
from flask_cors import CORS, cross_origin
#from trainingModel import trainModel
import flask_monitoringdashboard as dashboard
import json

os.putenv('LANG', 'en_US.UTF-8')
os.putenv('LC_ALL', 'en_US.UTF-8')

app = Flask(__name__)
dashboard.bind(app)
CORS(app)

@app.route("/", methods=['GET'])
@cross_origin()
def home():
    return render_template(r'C:/Users/Gautam/Desktop/waferFaultDetection/code/WaferFaultDetection_new/templates/index.html')

@app.route("/predict", methods=['GET','POST'])
@cross_origin()
def predictRouteClient():
    try:
        #if request.json is not None:
            #path = request.json['filepath']

            #pred_val = pred_validation(path) #object initialization

            #pred_val.prediction_validation() #calling the prediction_validation function

            #pred = prediction(path) #object initialization

            # predicting for dataset present in database
            #path,json_predictions = pred.predictionFromModel()
            #return Response("Prediction File created at !!!"  +str(path) +'and few of the predictions are '+str(json.loads(json_predictions) ))
        #elif request.form is not None:
            #path = request.form['filepath']

            #pred_val = pred_validation(path) #object initialization

            #pred_val.prediction_validation() #calling the prediction_validation function

            pred = prediction() #object initialization

            # predicting for dataset present in database
            path,json_predictions = pred.predictionFromModel()
            return Response("Prediction File created at !!!"  +str(path) +'and few of the predictions are '+str(json.loads(json_predictions) ))
        else:
            print('Nothing Matched')
    except ValueError:
        return Response("Error Occurred! a%s" %ValueError)
    except KeyError:
        return Response("Error Occurred! b%s" %KeyError)
    except Exception as e:
        return Response("Error Occurred! c%s" %e)


@app.route("/train", methods=['GET','POST'])
@cross_origin()
def trainRouteClient():

    try:
        #if request.json['folderPath'] is not None:
            #path = request.json['folderPath']

            #path = 'Training_Batch_Files'
            #train_valObj = train_validation(path) #object initialization

            #train_valObj.train_validation()#calling the training_validation function


            trainModelObj = trainModel() #object initialization
            trainModelObj.trainingModel() #training the model for the files in the table


    except ValueError:

        return Response("Error Occurred 12! %s" % ValueError)

    except KeyError:

        return Response("Error Occurred 23! %s" % KeyError)

    except Exception as e:

        return Response("Error Occurred 34! %s" % e)
    return Response("Training successfull!!")

#port = int(os.getenv("PORT",5000))

if __name__ == "__main__":
    host = '0.0.0.0'
    port = 8000
    httpd = simple_server.make_server(host, port, app)
    # print("Serving on %s %d" % (host, port))
    httpd.serve_forever()

Scheduler started


127.0.0.1 - - [30/Dec/2020 23:30:28] "GET /predict HTTP/1.1" 200 36
D:\ANACONDA\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  f"KMeans is known to have a memory leak on Windows "


Fitting 5 folds for each of 64 candidates, totalling 320 fits


D:\ANACONDA\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:30:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/5] END learning_rate=0.5, max_depth=3, n_estimators=10; total time=   0.8s
[23:30:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

...

[CV 5/5] END learning_rate=0.001, max_depth=20, n_estimators=200; total time=   0.9s
[23:39:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:39:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

D:\ANACONDA\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=10; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=10; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=10; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=10; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=10; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=50; total time=   0.3s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=50; total time=   0.3s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=50; total time=   0.3s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=50; total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_e

[CV 1/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=50; total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=50; total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=50; total time=   0.1s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=50; total time=   0.1s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=50; total time=   0.1s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=100; total time=   0.2s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=100; total time=   0.4s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=100; total time=   0.3s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=100; total time=   0.3s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=100; total time=   

127.0.0.1 - - [30/Dec/2020 23:40:19] "GET /train HTTP/1.1" 200 22
[2020-12-30 23:43:12,929] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "D:\ANACONDA\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\ANACONDA\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\ANACONDA\lib\site-packages\flask_cors\extension.py", line 161, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "D:\ANACONDA\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "D:\ANACONDA\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "D:\ANACONDA\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\ANACONDA\lib\site-packages\flask\app.py", line 1935, in dispatch_request
  